# TensorFlow 2.0

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

C:\Users\student\.conda\envs\deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\student\.conda\envs\deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\student\.conda\envs\deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\student\.conda\envs\deeplearning\lib\site-pack

## Hyperparameter Tunning
- 주로 import 다음에 첫 번째로 선언한다.

In [ ]:
num_epochs = 1
batch_size = 64

learning_rate = 0.001

dropout_rate = 0.7

input_shape = (28, 28, 1)
num_classes = 10

## Preprocess

In [2]:
(train_x, train_y), (test_x, test_y) = datasets.mnist.load_data()

In [3]:
# data 정제, nomalization
train_x = train_x[..., tf.newaxis]
test_x = test_x[..., tf.newaxis]

train_x = train_x / 255.
test_x = test_x / 255.

## Build Model

In [ ]:
inputs = layers.Input(input_shape)
net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [ ]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

## Training

In [ ]:
model.fit(train_x, train_y,   
             batch_size=batch_size, 
             shuffle=True)

model.evaluate(test_x, test_y, batch_size=batch_size)

# PyTorch

In [5]:
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# 이미지 관리를 위한 것
from torchvision import datasets, transforms

In [6]:
seed = 1

lr = 0.001
momentum = 0.5

batch_size = 64
test_batch_size = 64

epochs = 5

no_cuda = False # cuda가 있으면 T, cuda는 cpu로는 오래걸리는 data 분석을 더 빠르게 하기 위한 GPU
log_interval = 100 # 중간 log 보기 위한 것 > keras는 자동으로 나옴

## Model

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # in_channel, out_channel 둘 다 넣어줘야 함
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

## Preprocess

In [8]:
torch.manual_seed(seed)

use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}


# data를 가져오기 위한 것
# loader 안에서 전처리까지 처리
# tensorflow는 loader가 없어서 간단해보이만, 실제 data는 예쁘게 정리 안되어있음
# 그래서 loader로 전처리까지 할 수 있는 형식이 pytoch의 장점
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)



test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)

100.1%

Extracting ../data\MNIST\raw\train-images-idx3-ubyte.gz to ../data\MNIST\raw


113.5%

Extracting ../data\MNIST\raw\train-labels-idx1-ubyte.gz to ../data\MNIST\raw


100.4%

Extracting ../data\MNIST\raw\t10k-images-idx3-ubyte.gz to ../data\MNIST\raw


180.4%

Extracting ../data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ../data\MNIST\raw
Processing...
Done!


## Optimization

In [9]:
# tensorflow에서는 complie에 opt와 loss, metricx 다 쓰지만 pytoch는 opt 따로, traing에 loss와 metricx를 넣음
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

## Training

In [10]:
for epoch in range(1, epochs + 1):
    # Train Mode
    model.train()
    
    # epoch안에서 batch 사이즈만큼 반복
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()  # backpropagation 계산하기 전에 0으로 기울기 계산
        output = model(data)
        loss = F.nll_loss(output, target)  # https://pytorch.org/docs/stable/nn.html#nll-loss
        loss.backward()  # 계산한 기울기를 
        optimizer.step()  

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    
    # Test mode
    model.eval()  # batch norm이나 dropout 등을 train mode 변환
    test_loss = 0
    correct = 0
    with torch.no_grad():  # autograd engine, 즉 backpropagatin이나 gradient 계산 등을 꺼서 memory usage를 줄이고 속도를 높임
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()  # pred와 target과 같은지 확인
    
    print(test_loss)
    # batch들의 loss 평균을 구해서 이를 1 epoch의 loss로 생각함
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.300039
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.239658
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.133953
Train Epoch: 1 [19200/60000 (32%)]	Loss: 2.007274
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.656336
Train Epoch: 1 [32000/60000 (53%)]	Loss: 1.400260
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.864180
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.613125
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.615019
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.427777
4821.676996707916

Test set: Average loss: 0.4822, Accuracy: 8650/10000 (86%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.635707
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.361418
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.457257
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.394011
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.536249
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.258104
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.413024
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.174126
Train Epoch: 2 [51200/60000 (85%)